# Machine learning for genetic data

## Introduction

The goal of this practical session is to manipulate high-dimensional, low sample-size data that is typical of many genetic applications.

Here we will work with GWAS data from _Arabidopsis thaliana_, which is a plant model organism. The genotypes are hence described by **Single Nucleotide Polymorphisms, or SNPs**. Our goal will be to use this data to identify regions of the genome that can be linked with various growth and flowering traits (**phenotypes**).

## Data description

* `data/athaliana_small.X.txt` is the design matrix. As many rows as samples, as many columns as SNPs
* the SNPs are given (in order) in `data/athaliana_small.snps.txt`. 
* the samples are given (in order) in `data/athaliana.samples.txt`.

* the phenotypes are given in `data/phenotypes.pheno`. The first two columns give the sample's ID, and all following columns give a phenotype. The header gives the list of all phenotypes. In this session we will use "2W" and "4W", which give the number of days by which the plant grows to be 5 centimeters tall, after either two weeks ("2W") or four weeks ("4W") of vernalization (i.e. the seeds are kept at cold temperatures, similar to winter). Not all phenotypes are available for all samples.

* `data/athaliana.snps_by_gene.txt` contains, for each _A. thaliana_ SNP, the list of genes it is in or near to. (This can be several genes, as it is customary to use a rather large window to compute this, so as to capture potential cis-regulatory effects.)

* the feature network is in `data/athaliana_small.W.txt`. It has been saved as 3 arrays, corresponding to the row, col, and data attributes of a [scipy.sparse coo_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html).

## Loading the data

In [ ]:
%pylab inline

#### Read the list of SNP names

In [ ]:
with open('data/athaliana_small.snps.txt') as f:
    snp_names = f.readline().split()
    f.close()
print len(snp_names), snp_names[:10]

#### Read the list of sample names

In [ ]:
samples = list(np.loadtxt('data/athaliana.samples.txt', # file names
                         dtype=int)) # values are integers
print len(samples), samples[:10]

#### Load the design matrix (n samples x p SNPs)

In [ ]:
X = np.loadtxt('data/athaliana_small.X.txt',  # file names
               dtype='int') # values are integers

In [ ]:
n, p = X.shape

#### Load the 2W phenotype data

The first phenotype we will work with is called "2W". It describes the number of days required for the bolt height to reach 5 cm, at a temperature of 23°C under 16 hours of daylight per 24 hours, for seeds that have been vernalized for 2 weeks at 5°C (with 8 hours of daylight per 24 hours).

In [ ]:
import pandas as pd

In [ ]:
# TODO
# read phenotypes from phenotypes.pheno
# only keep samples that have a 2W phenotype. 

In [ ]:
# Restrict X to the samples with a 2W phenotype, in correct order
# X_2W[i] = X[samples.index(samples_with_phenotype[i])]
X_2W = X[np.array([samples.index(sample_id) \
                   for sample_id in samples_with_phenotype]), :]
n, p = X_2W.shape
print n, p

In [ ]:
# You can delete X now if you want, to free space
del X

## Split the data in a train and test set

We will set aside a test set, containing 20% of our samples, on which to evaluate the quality of our predictive models.

In [ ]:
from sklearn import model_selection

In [ ]:
X_2W_tr, X_2W_te, y_2W_tr, y_2W_te = \
    model_selection.train_test_split(X_2W, y_2W, test_size=0.2, 
                                     random_state=17)
print X_2W_tr.shape, X_2W_te.shape

## Visualize the phenotype

In [ ]:
h = plt.hist(y_2W_tr, bins=30)

## T-test

Let us start by running a statistical test for association of each SNP feature with the phenotype.

In [ ]:
import statsmodels.api as sm

In [ ]:
p = X_2W_tr.shape[1]

In [ ]:
pvalues = []
for snp_idx in range(p):
    # only look a the column corresponding at that SNP
    X_snp = X_2W_tr[:, snp_idx]
    # run a linear regression (with bias) between the phenotype and 
    # this SNP
    X_snp = sm.add_constant(X_snp)
    est = sm.regression.linear_model.OLS(y_2W_tr, X_snp)
    est2 = est.fit()
    # get the p-value from the model 
    pvalues.append(est2.pvalues[1])
pvalues = np.array(pvalues)

### Manhattan plot

The common way to visualize such results is by using a Manhattan plot: we will plot all SNPs on the x-axis, and on the y-axis we'll have the opposite of the log base 10 of the p-value. The lower the p-value, the higher the corresponding marker. 

We will also add a horizontal line that corresponds to the _threshold for significance_. Because we are testing multiple hypotheses, we need to lower our threshold accordingly. We will use __Bonferroni correction__ and divide the significance threshold (say, alpha=0.05) by the number of tests, that is, the number of SNPs p.

In [ ]:
plt.scatter(range(p), # x = SNP position
            -np.log10(pvalues)) # y = -log10 p-value 

# significance threshold according to Bonferroni correction
t = -np.log10(0.05/p)
plt.plot([0, p], [t, t])

# plot labels
plt.xlabel("feature")
plt.ylabel("-log10 p-value")
plt.xlim([0, p])

__What do you observe? Are any SNPs significantly associated with the phenotype? What genes are they in/near?__

## Linear regression 

In [ ]:
from sklearn import linear_model

In [ ]:
model_lr = linear_model.LinearRegression(fit_intercept=True)
model_lr.fit(X_2W_tr, y_2W_tr)

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(range(p), # x = SNP position
            model_lr.coef_)  # y = regression weights

plt.xlabel("SNP")
plt.ylabel("regression weight")
plt.xlim([0, p])

__What do you observe? How can you interpret these results? Do any of the SNPs strike you as having a strong influence on the phenotype?__

### Model predictive power

In [ ]:
from sklearn import metrics

In [ ]:
y_2W_lr_pred = model_lr.predict(X_2W_te)

print "Percentage of variance explained (using all SNPs): %.2f" % \
    metrics.explained_variance_score(y_2W_te, y_2W_lr_pred)

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_2W_te, y_2W_lr_pred)

plt.xlabel("true phenotype")
plt.ylabel("prediction")
plt.xlim([np.min(y_2W_te)-5, np.max(y_2W_te)+5])
plt.ylim([np.min(y_2W_te)-5, np.max(y_2W_te)+5])

## Lasso

In [ ]:
alphas = np.logspace(-4., 1., num=20)

In [ ]:
lasso = linear_model.Lasso(fit_intercept=True)
model_l1 = model_selection.GridSearchCV(lasso, 
                                        param_grid={'alpha': alphas}, 
                                        scoring='explained_variance')
model_l1.fit(X_2W_tr, y_2W_tr)

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(range(p), # x = SNP position
            model_l1.best_estimator_.coef_)  # y = regression weights

plt.xlabel("SNP")
plt.ylabel("lasso regression weight")
plt.xlim([0, p])

__How can you interpret these results? How many SNPs contribute to explaining the phenotype?__

In [ ]:
print "%d SNPs selected." % \
    np.nonzero(model_l1.best_estimator_.coef_)[0].shape

### Predictive power 

In [ ]:
y_2W_l1_pred = model_l1.best_estimator_.predict(X_2W_te)

print "Percentage of variance explained (using %d SNPs): %.2f" % \
     (np.nonzero(model_l1.best_estimator_.coef_)[0].shape[0], 
      metrics.explained_variance_score(y_2W_te, y_2W_l1_pred))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_2W_te, y_2W_l1_pred)

plt.xlabel("true phenotype")
plt.ylabel("prediction")
plt.xlim([np.min(y_2W_te)-0.05, np.max(y_2W_te)+0.05])
plt.ylim([np.min(y_2W_te)-0.05, np.max(y_2W_te)+0.05])

### Stability

__How stable is the set of selected SNPs, between the different rounds of cross-validation with optimal parameters?__

You can use [sklearn.metrics.jaccard_similarity_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_similarity_score.html), or implement Kuncheva's consistency index.

__Note:__ One could also contemplate using the Jaccard similarity (or another measure of consistency/stability/robustness) as a criterion to select the best hyperparameters. Pay attention, however, to the fact that hyperparameters selecting no features at all or all the features will have very good consistency.

## Elastic net

One solution to make the lasso more stable is to use a combination of the l1 and l2 regularizations.

We are now minimizing the loss + a linear combination of an l1-norm and an l2-norm over the regression weights. This imposes sparsity, but encourages correlated features to be selected together, where the lasso would tend to pick only one (at random) of a group of correlated features.

The elastic net is implemented in scikit-learn's [linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet).

In [ ]:
# Parameters grid
alphas = np.logspace(-4., 1., num=15)
ratios = np.linspace(0.5, 1., num=3)

In [ ]:
enet = linear_model.ElasticNet(fit_intercept=True)
model_l1l2 = model_selection.GridSearchCV(enet, 
                                        param_grid={'alpha': alphas, 
                                                    'l1_ratio': ratios}, 
                                        scoring='explained_variance')
model_l1l2.fit(X_2W_tr, y_2W_tr)

In [ ]:
plt.figure(figsize=(6, 4))
plt.scatter(range(p), # x = SNP position
            model_l1l2.best_estimator_.coef_)  # y = regression weights

plt.xlabel("SNP")
plt.ylabel("elastic net regression weight")
plt.xlim([0, p])

__How can you interpret these results? How many SNPs contribute to explaining the phenotype?__

In [ ]:
print "%d SNPs selected." % \
    np.nonzero(model_l1l2.best_estimator_.coef_)[0].shape

### Predictive power 

In [ ]:
y_2W_l1l2_pred = model_l1.best_estimator_.predict(X_2W_te)

print "Percentage of variance explained (using %d SNPs): %.2f" % \
     (np.nonzero(model_l1l2.best_estimator_.coef_)[0].shape[0], 
      metrics.explained_variance_score(y_2W_te, y_2W_l1_pred))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_2W_te, y_2W_l1l2_pred)

plt.xlabel("true phenotype")
plt.ylabel("prediction")
plt.xlim([np.min(y_2W_te)-0.05, np.max(y_2W_te)+0.05])
plt.ylim([np.min(y_2W_te)-0.05, np.max(y_2W_te)+0.05])

### Stability

__How stable is the set of selected SNPs, between the different rounds of cross-validation with optimal parameters?__

## Stability selection with the Lasso

__Use a randomized procedure to stabilize the lasso__

[sklearn.linear_model.RandomizedLasso.html](http://scikit-learn.org/0.18/modules/generated/sklearn.linear_model.RandomizedLasso.html#sklearn.linear_model.RandomizedLasso) + [User Guide](http://scikit-learn.org/0.18/auto_examples/linear_model/plot_sparse_recovery.html)

## Network-constrained lasso

This is not implemented in scikit-learn, so we'll need to create our own estimator.

It turns out that it is possible to transform the network-constrained Lasso problem into a Lasso problem: follow [the original paper](https://academic.oup.com/bioinformatics/article/24/9/1175/206444) (pdf also available [here](http://www.stat.purdue.edu/~doerge/BIOINFORM.D/FALL10/Li_and_Li_2008_Bioinformatics.pdf) and the note in section C of [the supplementary material of Sugiyama et al. (2014)](http://cazencott.info/dotclear/public/publications/sugiyama2014_supp.pdf) to replace the eigen-decomposition of the graph Laplacian with the graph incidence matrix.

Follow the [documentation](http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator) or this [blog post](http://danielhnyk.cz/creating-your-own-estimator-scikit-learn/) to create a scikit-learn estimator.

Be careful: the computations might require a lot of RAM.

### Load the network

In [ ]:
from scipy import sparse

In [ ]:
w_saved = np.loadtxt('data/athaliana_small.W.txt')

In [ ]:
W = sparse.coo_matrix((w_saved[2, :], (np.array(w_saved[0, :], dtype=int), 
                                       np.array(w_saved[1, :], dtype=int))), 
                      shape=(p, p))

### Build the incidence matrix

In [ ]:
# Compute node degrees 
degrees = np.zeros((p, ))
for vertex in W.row:
    degrees[vertex] += 2

In [ ]:
tim = sparse.lil_matrix((W.row.shape[0], p))
for ix, edge in enumerate(W.data):
    tim[ix, W.row[ix]] = np.sqrt(edge / degrees[W.row[ix]])
    tim[ix, W.col[ix]] = - np.sqrt(edge / degrees[W.col[ix]])

### Create the network-constrained Lasso class

In [ ]:
from sklearn import base, linear_model

In [ ]:
class ncLasso(base.BaseEstimator, base.RegressorMixin):
    def __init__(self, transposed_incidence=None, lambda1=1.0, lambda2=1.0):
        self.transposed_incidence = transposed_incidence # sparse matrix
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        
    def fit(self, X, y):       
        alpha = self.lambda1/(np.sqrt(self.lambda2+1.))
        self.lasso = linear_model.Lasso(fit_intercept=True, alpha=alpha)
        
        
        y_new = np.hstack((y, np.zeros((self.transposed_incidence.shape[0], ))))
        print y_new.shape, X.shape
        X_new = 1/(np.sqrt(self.lambda2+1)) * sparse.vstack((X, np.sqrt(self.lambda2)*\
                                                    self.transposed_incidence))
        
        
        self.lasso.fit(X_new, y_new)
        self.coef_ = self.lasso.coef_[:X.shape[1]]/(np.sqrt(self.lambda2+1))
        return self
        
        
    def predict(self, X, y=None):
        return self.lasso.predict(X)
    
    
    def score(self, X, y=None):
        return self.lasso.score(X, y)                                        

__Use the network-constrained Lasso on the data. What do you observe?__

If you want to use SConES/sfan, see [github/chagaz/sfan](https://github.com/chagaz/sfan). The StructuredSparsity notebook can help.

## Multi-task feature selection

1) Repeat the previous analysis for the 4W phenotype. It is very similar to the 2W phenotype, except that the seeds have been vernelized for 4 weeks. 

2) It is not unreasonable to expect the genomic regions driving both those phenotypes to be (almost) the same. Use the multi-task version of the Lasso, ENet, or ncLasso algorithms to analyzed both phenotypes simultaneously.

Use [sklearn.linear_model.MultiTaskLasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.MultiTaskLasso.html#sklearn.linear_model.MultiTaskLasso) + [User Guide](http://scikit-learn.org/stable/auto_examples/linear_model/plot_multi_task_lasso_support.html)